**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

- при изменении процента акции необходимо добавить предобработку по времени, соответственно предлогаемый товар будет меняться
- если юзер новый, можно предложить для начала просто топ товаров, а после первых покупок уже фильтровать
- если необходимо продать товар дороже, то предложить аналогичный что ранее покупал, только с большей маржей, дороже процентов на 10 или предлогать доп товар который обычно покупают с купленым товаром.
Например, пользователь ранее покупал стиральный порошок. Рекомендовать порошек в биоразлогаемой упаковке(сейчас это в тренде и стоит дороже), так же можно предложить кондиционер для белья, так как его обычно берет с порошком